In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt  # Grafik ve çizim
from collections import Counter  # Etiket sayılarını saymak için
from skimage import io   # Görüntüleri dosyadan okumak için
from skimage.color import rgb2gray # Görüntüleri gri tonlamaya dönüştürmek için
from skimage import filters  # Görüntü üzerinde filtre uygulamak için
from skimage import exposure  # Görüntü parlaklık işlemleri için
from tqdm import tqdm    ## Kod çalışırken ekranda ilerleme çubuğu gösterir.
import cv2  # Görüntü işleme ve çekim açısı tespiti için
import PIL.Image  # Görüntü meta verisi için
from PIL.ExifTags import TAGS  # EXIF meta verilerini okumak için
import math  # Matematik işlemleri için
import shutil  # Dosya kopyalama işlemleri için

class ImageAnalyzer:
    def __init__(self):
        self.image_directory = None
        self.filtered_directory = None  # Filtrelenmiş görüntüler için klasör
        self.image_files = []
        self.results = {
            'blurriness': [],
            'brightness': [],
            'contrast': [],
            'sizes': [],
            'formats': [],
            'file_sizes': [],
            'angles': []  # EKLENDİ: Çekim açılarını saklamak için
        }
        # Etiket bilgilerini saklayacak veri yapıları
        self.labels = []
        self.label_counts = {}
        
        # EKLENDİ: Meta veri ve gizli bilgileri saklamak için yapılar
        self.metadata = {}
        self.hidden_data = {}
        self.filtered_images = []  # EKLENDİ: Filtrelenmiş görüntüleri tutacak liste
    
    # Kullanıcıdan klasör yolunu al
    def get_image_directory(self):
        while True:
            image_directory = input("📁 Lütfen görüntülerin bulunduğu klasörün yolunu girin: ")
            if os.path.exists(image_directory):
                self.image_directory = image_directory
                # EKLENDİ: Filtrelenmiş görüntüler için klasör oluştur
                self.filtered_directory = os.path.join(os.path.dirname(image_directory), 
                                                    os.path.basename(image_directory) + "_filtered")
                return image_directory
            else:
                print("❌ Hata: Klasör bulunamadı! Lütfen doğru bir yol girin.")
    
    # Görüntü bulanıklığını ölç
    def measure_blurriness(self, img):
        gray = rgb2gray(img)    # Renkli görüntüyü siyah-beyaz yapar
        laplacian = filters.laplace(gray) # Görüntüdeki kenarları bulur, kenarlar belirsiz ise görüntü bulanıktır
        variance = np.var(laplacian) # Görüntüdeki değişimi, varyansı ölçer
        return variance   # Varyans düşükse bulanıktır
    
    # Görüntü parlaklık ve kontrastı ölç
    def brightness_contrast(self, img):
        gray = rgb2gray(img)
        brightness = np.mean(gray)  # Görüntünün ortalama parlaklığı
        contrast = np.std(gray)  # Kontrast farlılığını ölçer
        return brightness, contrast
    
    # Renk dağılımını analiz et
    def color_distribution(self, img):
        colors = ("red", "green", "blue")
        plt.figure(figsize=(10, 4))
        plt.title("🎨 Renk Dağılımı")
        for i, color in enumerate(colors):
            hist, bins = np.histogram(img[:, :, i], bins=256, range=(0, 1))  # her renk kanalı için histogram (yani renk değerlerinin dağılımı) hesaplıyoruz.
            plt.plot(bins[:-1], hist, color=color)
        plt.xlim([0, 1])
        plt.show()
    #Böylece bir görselin içinde hangi rengin ne kadar olduğunu görürüz
    
    # Dosya veya klasör adından etiket çıkarma fonksiyonu
    def extract_label(self, file_path):
        """
        Dosya yolundan veya adından etiket bilgisini çıkarır.
        Bu fonksiyon, veri setinizin yapısına bağlı olarak özelleştirilebilir.
        """
        # Örnek 1: Eğer görüntüler "sınıf_xxx.jpg" formatında ise
        filename = os.path.basename(file_path)
        if '_' in filename:
            return filename.split('_')[0]
            
        # Örnek 2: Eğer görüntüler sınıf isimli klasörler içinde ise
        parent_dir = os.path.basename(os.path.dirname(file_path))
        if parent_dir != os.path.basename(self.image_directory):
            return parent_dir
            
        # Varsayılan: Etiket bulunamadı
        return "unlabeled"
    
    # Etiket istatistiklerini gösterme fonksiyonu
    def show_label_statistics(self):
        """Veri setindeki etiketlerin dağılımını görselleştirir."""
        if not self.label_counts:
            print("❌ Etiket bilgisi bulunamadı!")
            return
            
        # Etiket sayılarını yazdır
        print(f"\n📊 ETİKET İSTATİSTİKLERİ")
        print(f"🏷️ Toplam Etiket Sayısı: {len(self.labels)}")
        print(f"🔤 Benzersiz Etiket Sınıfları: {len(self.label_counts)}")
        
        # Sınıf dağılımını göster
        print("\n📋 Etiket Sınıflarının Dağılımı:")
        for label, count in sorted(self.label_counts.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / len(self.labels)) * 100
            print(f"  {label}: {count} görüntü ({percentage:.1f}%)")
            
        # Görselleştirme
        plt.figure(figsize=(10, 6))
        plt.bar(self.label_counts.keys(), self.label_counts.values(), color='skyblue')
        plt.title('Veri Setindeki Etiket Dağılımı')
        plt.xlabel('Etiket Sınıfları')
        plt.ylabel('Görüntü Sayısı')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    # EKLENDİ: Görüntü meta verilerini çıkarma fonksiyonu
    def extract_metadata(self, img_path):
        """Görüntü dosyasından meta verileri (EXIF bilgileri) çıkarır."""
        metadata = {}
        try:
            with PIL.Image.open(img_path) as img:
                # EXIF bilgilerini al
                exif_data = img._getexif()
                if exif_data:
                    for tag_id, value in exif_data.items():
                        tag = TAGS.get(tag_id, tag_id)
                        metadata[tag] = value
                        
                # Görüntü temel özellikleri
                metadata['Format'] = img.format
                metadata['Mode'] = img.mode
                metadata['Size'] = img.size
        except Exception as e:
            metadata['error'] = str(e)
            
        return metadata
    
    # EKLENDİ: Görüntüdeki gizli veri tespiti
    def detect_hidden_data(self, img_path):
        """Görüntüdeki potansiyel gizli verileri tespit eder (steganografi)."""
        hidden_data = {}
        
        try:
            # Basit LSB (Least Significant Bit) steganografi kontrolü
            img = cv2.imread(img_path)
            if img is None:
                return {'error': 'Görüntü okunamadı'}
                
            # LSB analizi - basit bir yaklaşım
            b, g, r = cv2.split(img)
            lsb_b = b % 2  # En düşük anlamlı bit
            lsb_g = g % 2
            lsb_r = r % 2
            
            # LSB'lerin dağılımı - düzensizlik steganografi işareti olabilir
            lsb_entropy_b = np.sum(lsb_b) / lsb_b.size
            lsb_entropy_g = np.sum(lsb_g) / lsb_g.size
            lsb_entropy_r = np.sum(lsb_r) / lsb_r.size
            
            hidden_data['LSB_Distribution'] = {
                'Blue': lsb_entropy_b,
                'Green': lsb_entropy_g,
                'Red': lsb_entropy_r
            }
            
            # Basit bir steganografi tespit yöntemi
            # 0.45-0.55 aralığında olması doğal bir görüntü için beklenenden farklı olabilir
            suspicious = False
            for channel, value in hidden_data['LSB_Distribution'].items():
                if 0.45 <= value <= 0.55:
                    suspicious = True
                    
            hidden_data['Suspicious'] = suspicious
            
        except Exception as e:
            hidden_data['error'] = str(e)
            
        return hidden_data
    
    # EKLENDİ: Görüntü çekim açısı tespiti
    def detect_image_angle(self, img_path):
        """Görüntünün çekim açısını tahmin eder."""
        try:
            # EXIF verilerinden açı bilgisini kontrol et
            metadata = self.extract_metadata(img_path)
            if 'Orientation' in metadata:
                # EXIF Orientation değeri açı bilgisi içerebilir
                orientation = metadata['Orientation']
                # Burada EXIF verilerinden açıyı çıkarabiliriz, ancak basit bir tahmin yapalım
                
            # Görüntü üzerinde line detection yaparak açı tespiti
            img = cv2.imread(img_path)
            if img is None:
                return 0  # Varsayılan değer
                
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150, apertureSize=3)
            
            # Hough Line dönüşümü ile çizgileri tespit et
            lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
            
            angles = []
            if lines is not None:
                for line in lines:
                    rho, theta = line[0]
                    # Açıyı dereceye çevir
                    angle_deg = theta * 180 / np.pi
                    # Yatay açıyı bul (0-90 arası)
                    if angle_deg > 90:
                        angle_deg = 180 - angle_deg
                    angles.append(angle_deg)
                
                if angles:
                    # En sık görülen açıyı al
                    angle_estimate = np.median(angles)
                    return angle_estimate
            
            # Açı tespit edilemezse varsayılan değer
            return 80  # Çoğu görüntü yatay olduğu için makul bir varsayılan değer
            
        except Exception as e:
            print(f"Açı tespiti hatası: {e}")
            return 80  # Hata durumunda varsayılan değer
    
    def load_images(self):
        if not self.image_directory:
            self.get_image_directory()
        
        # Arama için kullanılacak dosya tiplerini genişlettim (jpg, jpeg, png)
        self.image_files = [f for f in os.listdir(self.image_directory) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        if len(self.image_files) == 0:
            print("❌ Klasörde uygun formatta görüntü bulunamadı.")
            return False
        print(f"📈 Toplam Görüntü Sayısı: {len(self.image_files)}")
        return True
    
    def analyze_image(self, img_file):
        img_path = os.path.join(self.image_directory, img_file)
        
        try:
            img = io.imread(img_path)  
            
            # Boyut ve format bilgileri
            self.results['sizes'].append(img.shape)
            self.results['formats'].append(img.shape[-1] if len(img.shape) == 3 else 1)
            
            # Bulanıklık
            blurriness = self.measure_blurriness(img)
            self.results['blurriness'].append(blurriness)
            
            # Parlaklık ve kontrast
            brightness, contrast = self.brightness_contrast(img)
            self.results['brightness'].append(brightness)
            self.results['contrast'].append(contrast)
            
            # Görüntü boyutu için(MB)
            img_size_bytes = img.nbytes
            self.results['file_sizes'].append(img_size_bytes / (1024 * 1024))
            
            # Etiket çıkarma ve sayma
            label = self.extract_label(img_path)
            self.labels.append(label)
            
            # EKLENDİ: Meta veri çıkarma
            self.metadata[img_file] = self.extract_metadata(img_path)
            
            # EKLENDİ: Gizli veri tespiti
            self.hidden_data[img_file] = self.detect_hidden_data(img_path)
            
            # EKLENDİ: Çekim açısı tespiti
            angle = self.detect_image_angle(img_path)
            self.results['angles'].append(angle)
            
            # EKLENDİ: Çekim açısına göre filtreleme (70-90 derece aralığı, kapalı aralık)
            if 70 <= angle <= 90:
                self.filtered_images.append(img_file)
            
            return img
            
        except Exception as e:
            print(f"❌ '{img_file}' analiz edilirken hata: {e}")
            # Hata durumunda varsayılan değerler ekle
            self.results['sizes'].append((0, 0, 0))
            self.results['formats'].append(0)
            self.results['blurriness'].append(0)
            self.results['brightness'].append(0)
            self.results['contrast'].append(0)
            self.results['file_sizes'].append(0)
            self.results['angles'].append(0)
            self.labels.append("error")
            return None
    
    # EKLENDİ: Filtrelenmiş görüntüleri yeni klasöre kopyala
    def save_filtered_images(self):
        """70-90 derece aralığındaki görüntüleri yeni bir klasöre kopyalar."""
        if not self.filtered_images:
            print("⚠️ Filtreleme kriterlerine uyan görüntü bulunamadı.")
            return
            
        # Hedef klasörü oluştur (yoksa)
        if not os.path.exists(self.filtered_directory):
            os.makedirs(self.filtered_directory)
            
        # Görüntüleri kopyala
        for img_file in tqdm(self.filtered_images, desc="📋 Filtrelenmiş görüntüler kopyalanıyor"):
            src_path = os.path.join(self.image_directory, img_file)
            dst_path = os.path.join(self.filtered_directory, img_file)
            shutil.copy2(src_path, dst_path)
            
        print(f"✅ {len(self.filtered_images)} görüntü '{self.filtered_directory}' klasörüne kopyalandı.")
        
    # EKLENDİ: Meta verileri ve gizli bilgileri göster
    def show_metadata_analysis(self):
        """Görüntülerdeki meta verilerin ve gizli bilgilerin analizini gösterir."""
        if not self.metadata:
            print("❌ Meta veri bilgisi bulunamadı!")
            return
            
        print("\n📊 GÖRÜNTÜ META VERİ ANALİZİ")
        
        # Yaygın meta veri alanlarını bul
        common_tags = set()
        for data in self.metadata.values():
            common_tags.update(data.keys())
            
        print(f"🔎 Tespit Edilen Meta Veri Alanları: {len(common_tags)}")
        print("En yaygın meta veri alanları:")
        
        # Her bir meta veri alanının kaç görüntüde bulunduğunu say
        tag_counts = {}
        for tag in common_tags:
            count = sum(1 for data in self.metadata.values() if tag in data)
            tag_counts[tag] = count
            
        # En yaygın 10 alanı göster
        for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
            percentage = (count / len(self.metadata)) * 100
            print(f"  {tag}: {count} görüntüde ({percentage:.1f}%)")
            
        # Gizli veri analizi
        print("\n🔍 GİZLİ VERİ ANALİZİ")
        suspicious_count = sum(1 for data in self.hidden_data.values() 
                               if 'Suspicious' in data and data['Suspicious'])
        
        if suspicious_count > 0:
            print(f"⚠️ Potansiyel gizli veri içeren görüntü sayısı: {suspicious_count}")
            print("Şüpheli görüntüler:")
            for img_file, data in self.hidden_data.items():
                if 'Suspicious' in data and data['Suspicious']:
                    print(f"  - {img_file}")
        else:
            print("✅ Tespit edilen şüpheli gizli veri yok.")
    
    # Ana EDA fonksiyonu
    def run_image_eda(self):
        if not self.load_images():
            return
        
        # Tüm görüntüleri analiz et
        for img_file in tqdm(self.image_files, desc="🔍 Görüntüler Analiz Ediliyor"):
            self.analyze_image(img_file)
        
        # Etiket sayılarını hesapla
        self.label_counts = Counter(self.labels)
            
        # Sonuçları yazdır
        self.print_results()
        
        # Etiket istatistiklerini göster
        self.show_label_statistics()
        
        # EKLENDİ: Meta veri analizini göster
        self.show_metadata_analysis()
        
        # EKLENDİ: Çekim açısı dağılımını göster
        self.show_angle_distribution()
        
        # EKLENDİ: Filtrelenmiş görüntüleri kaydet
        self.save_filtered_images()
    
    def print_results(self):
        # Genel bilgiler
        print(f"\n📊 GENEL İSTATİSTİKLER")
        print(f"📏 Ortalama Görüntü Boyutu: {np.mean([s[0]*s[1] for s in self.results['sizes']]):.2f} piksel")
        print(f"🔵 Ortalama Parlaklık: {np.mean(self.results['brightness']):.2f}")
        print(f"⚫ Ortalama Kontrast: {np.mean(self.results['contrast']):.2f}")
        print(f"🔍 Ortalama Bulanıklık: {np.mean(self.results['blurriness']):.2f}")
        print(f"📦 Ortalama Görüntü Dosya Boyutu: {np.mean(self.results['file_sizes']):.2f} MB")
        
        # EKLENDİ: Filtreleme bilgileri
        filtered_count = len(self.filtered_images)
        filtered_percentage = (filtered_count / len(self.image_files)) * 100 if self.image_files else 0
        print(f"\n📐 ÇEKİM AÇISI FİLTRELEME")
        print(f"✅ 70-90 derece aralığındaki görüntü sayısı: {filtered_count} ({filtered_percentage:.1f}%)")
        print(f"❌ Filtrelenen görüntü sayısı: {len(self.image_files) - filtered_count}")
    
    # EKLENDİ: Çekim açısı dağılımını göster
    def show_angle_distribution(self):
        """Veri setindeki görüntülerin çekim açısı dağılımını görselleştirir."""
        if not self.results['angles']:
            print("❌ Çekim açısı bilgisi bulunamadı!")
            return
            
        angles = self.results['angles']
        
        plt.figure(figsize=(10, 6))
        plt.hist(angles, bins=18, range=(0, 90), color='skyblue', edgecolor='black')
        plt.title('Görüntü Çekim Açısı Dağılımı')
        plt.xlabel('Açı (Derece)')
        plt.ylabel('Görüntü Sayısı')
        
        # 70-90 derece aralığını vurgula
        plt.axvspan(70, 90, alpha=0.2, color='green', label='Filtrelenen Aralık (70-90°)')
        plt.legend()
        
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    
    def show_color_distribution_for_image(self, index):
        """Belirli bir görüntü için renk dağılımını gösterir"""
        if 0 <= index < len(self.image_files):
            img_path = os.path.join(self.image_directory, self.image_files[index])
            img = io.imread(img_path)
            self.color_distribution(img)
        else:
            print(f"❌ Hata: Geçersiz görüntü indeksi. 0-{len(self.image_files)-1} arası bir değer girin.")
    
    # Belirli bir sınıfa ait görüntülerin analizi
    def analyze_class(self, class_name):
        """Belirli bir sınıfa ait görüntülerin detaylı analizini yapar"""
        if class_name not in self.label_counts:
            print(f"❌ '{class_name}' sınıfı veri setinde bulunamadı.")
            return
            
        class_indices = [i for i, label in enumerate(self.labels) if label == class_name]
        class_blurriness = [self.results['blurriness'][i] for i in class_indices]
        class_brightness = [self.results['brightness'][i] for i in class_indices]
        class_contrast = [self.results['contrast'][i] for i in class_indices]
        class_angles = [self.results['angles'][i] for i in class_indices]
        
        # EKLENDİ: Sınıf analizi genişletildi
        filtered_class_count = sum(1 for i in class_indices if 70 <= self.results['angles'][i] <= 90)
        
        print(f"\n🔍 '{class_name}' SINIFI ANALİZİ")
        print(f"📈 Görüntü Sayısı: {len(class_indices)}")
        print(f"🔵 Ortalama Parlaklık: {np.mean(class_brightness):.2f}")
        print(f"⚫ Ortalama Kontrast: {np.mean(class_contrast):.2f}")
        print(f"🔍 Ortalama Bulanıklık: {np.mean(class_blurriness):.2f}")
        print(f"📐 Ortalama Çekim Açısı: {np.mean(class_angles):.2f}°")
        print(f"✅ 70-90° aralığındaki görüntü sayısı: {filtered_class_count} ({filtered_class_count/len(class_indices)*100:.1f}%)")

# Kullanım örneği
if __name__ == "__main__":
    analyzer = ImageAnalyzer()
    analyzer.run_image_eda()
    
    # Eğer veri setinizde etiketler varsa, belirli bir sınıfı analiz etmek için:
    # analyzer.analyze_class("kedi")
    
    # Opsiyonel: İlk görüntü için renk dağılımını göster
    # analyzer.show_color_distribution_for_image(0)

ModuleNotFoundError: No module named 'tqdm'